#Instalacja bibliotek

In [ ]:
#instalacja biblioteki Transformers.
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.8 MB/s eta 0:00:00


In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
import io
import os
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW,
                          get_linear_schedule_with_warmup,
                          GPT2LMHeadModel)

In [ ]:
import pandas as pd

In [ ]:
import os

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Połaczenie z Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
from google.colab import files

#Folder danych

In [ ]:
img_folder = "/content/gdrive/MyDrive/Colab Notebooks/PW PROJEKT/train/"

#Parametry

In [ ]:
LOG_EVERY_N = 20

#DETR

Import modelu DETR w celu detekcji obiektów na obrazie.

https://huggingface.co/facebook/detr-resnet-50

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

In [ ]:
processor_detr = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model_detr = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch

device = torch.device("cpu")

if torch.cuda.is_available():
   print("Training on GPU")
   device = torch.device("cuda:0")

Training on GPU


In [ ]:
model_detr.to(device)

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

#VisionEncoderDecoderModel

Zaimportowanie VisionEncoderDecoderModel w celu opisu fragmentu obrazu.

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

In [ ]:
model_vit = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor_vit = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer_vit = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_vit.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [ ]:
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

#GPT2LMHead

Import modelu językowego GPT-2 oraz wytrenowanego soft-promptu.

In [ ]:
sf_file_name = "/content/gdrive/MyDrive/Colab Notebooks/PW PROJEKT/csvs/dets/soft_prompt/soft_prompt_10_e3_lr0.001.pt"

In [ ]:
sf = torch.load(sf_file_name)

In [ ]:
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

In [ ]:
gpt_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
gpt_model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
def classify_qs(question, sentence, model, tokenizer, soft_prompt):
  #klasyfikacja, czy zdanie "sentence" zawiera odpowiedź na pytanie "question"
  text_wrapped = f"Question: {question}; Sentence: {sentence}; Answer:"
  text = tokenizer.encode(text_wrapped)
  text_embeds = model.transformer.wte.weight[text, :]
  embeds = torch.cat((soft_prompt.to(device), text_embeds.to(device)), 0).unsqueeze(0).to(device)

  outputs = model(inputs_embeds=embeds)
  softmax_logits = torch.softmax(outputs.logits, dim=-1)
  predicted_label = softmax_logits.detach().argmax(axis=-1).flatten().tolist()[-1]
  decoded = tokenizer.decode(predicted_label)

  return decoded

In [ ]:
question = "Where can i put my flowers"
sentence = "a white vase filled with flowers on top of a table"

g = classify_qs(question, sentence, gpt_model, gpt_tokenizer, sf)

In [ ]:
g

'yes'

#Funkcje pomocnicze

In [ ]:
def predict_step(image_paths):
  #opisuje obrazy z podanej listy ścieżek
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor_vit(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model_vit.generate(pixel_values, **gen_kwargs)

  preds = tokenizer_vit.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

In [ ]:
def describe_img(df, log_every_n=LOG_EVERY_N):
  df["description"] = ""

  for index, row in df.iterrows():
    if os.path.exists(row["det_obj_img_name"]):
      desc = predict_step([row["det_obj_img_name"]])[0]
      df.at[index,"description"] = desc
      if (index % LOG_EVERY_N) == 0:
        print(f"Image {index} ({row['det_obj_img_name']}) has been described as: {desc}.")
    else:
      print(f"File {row['det_obj_img_name']} does not exist.")
  df.to_csv(f"/content/val_desc.csv", index=False)
  df.to_csv(CSV_FOLDER, index=False)
  files.download(f"/content/val_desc.csv")

  return df

In [ ]:
def remove_img(img_folder):
  for img in os.listdir(img_folder):
    img_path = f"{img_folder}/{img}"
    os.remove(img_path)

In [ ]:
DEST_IMG_FOLDER = f"/content/gdrive/MyDrive/Colab Notebooks/PW PROJEKT/val/tmp"

In [ ]:
def crop_detected_object_and_save(source_img_name, image, det_obj_num, box, dest_img_folder = DEST_IMG_FOLDER):
  if not os.path.exists(dest_img_folder):
    os.mkdir(dest_img_folder)
  img = image.crop((box[0], box[1], box[2], box[3]))
  if source_img_name[-4:]==".jpg":
    source_img_name = source_img_name[:-4]
  #os.mkdir(f"{dest_img_folder}")
  name = f"{dest_img_folder}/{source_img_name}_{det_obj_num}.jpg"
  img = img.save(name)
  return name


In [ ]:
def detect_objects(img_name, treshold=0.3):
  df = pd.DataFrame(columns = ["img_name", "left", "top", "right", "bottom", "label", "score", "det_obj_img_name"])
  img = img_folder + img_name
  image = Image.open(img)
  image = image.convert('RGB')
  inputs = processor_detr(images=image, return_tensors="pt")
  inputs.to(device)
  outputs = model_detr(**inputs)
  target_sizes = torch.tensor([image.size[::-1]])
  results = processor_detr.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=treshold)[0]
  idx = 0

  for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    fbox = [round(i, 2) for i in box.tolist()]
    flabel = model_detr.config.id2label[label.item()]
    fscore = round(score.item(), 3)
    #save cropped objects as .jpg
    box = box.detach().cpu().numpy()
    det_obj_img_name = crop_detected_object_and_save(img_name, image, idx, box)

    df = df.append({
        "img_name":img_name,
        "left":fbox[0],
        "top":fbox[1],
        "right":fbox[2],
        "bottom":fbox[3],
        "label":flabel,
        "score":fscore,
        "det_obj_img_name":det_obj_img_name},
        ignore_index = True)
    idx+=1
  return df

In [ ]:
def calculate_iou(ground_truth_coord, detected_object_coord):
  gt_x1 = ground_truth_coord[0]
  gt_y1 = ground_truth_coord[1]
  gt_x2 = ground_truth_coord[2]
  gt_y2 = ground_truth_coord[3]

  det_x1 = detected_object_coord[0]
  det_y1 = detected_object_coord[1]
  det_x2 = detected_object_coord[2]
  det_y2 = detected_object_coord[3]

  int_x1 = max(gt_x1, det_x1)
  int_y1 = max(gt_y1, det_y1)
  int_x2 = min(gt_x2, det_x2)
  int_y2 = min(gt_y2, det_y2)

  intersection_width = int_x2 - int_x1
  intersection_height = int_y2 - int_y1

  if intersection_width >=0 and intersection_height >=0:
    intersection = (intersection_width)*(intersection_height)
  else:
    intersection = 0

  ground_truth_area = (abs(gt_x2 - gt_x1))*(abs(gt_y2 - gt_y1))
  detected_object_area = (abs(det_x2 - det_x1))*(abs(det_y2 - det_y1))

  union = ground_truth_area + detected_object_area - intersection

  if union != 0.0:
    iou = intersection/union
  else:
    iou = 0.0

  return iou

#Walidacja

In [ ]:
VAL_FOLDER = "/content/gdrive/MyDrive/Colab Notebooks/PW PROJEKT/val"

In [ ]:
#folder z przygotowanymi danymi walidacyjnymi w formacie .csv
CSV_FOLDER = "/content/gdrive/MyDrive/Colab Notebooks/PW PROJEKT/csvs/dets/raw_data/output"

In [ ]:
csv_test = pd.read_csv(f"{CSV_FOLDER}/test_data.csv")

In [ ]:
csv_test.head()

,image,width,height,left,top,right,bottom,question,img_name,left_det,top_det,right_det,bottom_det,label,score,det_obj_img_name,iou,binary_label,description
0,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,427,182,257,190,268,What is the animal using to breathe?,000000528882.jpg,183.28,104.81,625.09,375.26,bear,1.000,/content/gdrive/MyDrive/Colab Notebooks/PW PRO...,0.000619,no,a large brown bear standing on top of a lush g...
1,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,480,640,214,123,266,159,What do I use to proctect my eyes?,000000411744.jpg,0.02,567.96,478.59,625.35,snowboard,0.998,/content/gdrive/MyDrive/Colab Notebooks/PW PRO...,0.000000,no,a person on a surfboard in the water
2,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,480,360,274,82,451,212,Where can i watch netflix?,000000047478.jpg,274.47,82.62,449.87,211.71,tv,0.999,/content/gdrive/MyDrive/Colab Notebooks/PW PRO...,0.984024,yes,a flat screen tv sitting on top of a wooden stand
3,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,480,440,354,517,371,What do we use to slide down a hill?,000000022017.jpg,441.55,357.72,515.35,370.81,skis,0.990,/content/gdrive/MyDrive/Colab Notebooks/PW PRO...,0.738000,yes,a pair of people riding skis down a snow cover...
4,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,478,208,19,543,202,Where can i watch memes?,000000414911.jpg,198.36,21.29,561.22,210.11,tv,0.986,/content/gdrive/MyDrive/Colab Notebooks/PW PRO...,0.873784,yes,a computer screen with a cartoon character on it


In [ ]:
csv = csv_test[['image', 'width', 'height', 'left', 'top', 'right', 'bottom', 'question', 'img_name']].copy()

In [ ]:
det_df = pd.DataFrame(columns =
 [
    "image",
    "width",
    "height",
    "left",
    "top",
    "right",
    "bottom",
    "question",
    "img_name",
    "left_det",
    "top_det",
    "right_det",
    "bottom_det",
    "label",
    "score",
    "det_obj_img_name",
    "iou",
    "binary_label",
    "description"
    ])

In [ ]:
for index, row in csv.iterrows():
  det_tmp_df = pd.DataFrame(columns = ["image","width","height","left","top","right","bottom","question","img_name","left_det","top_det","right_det","bottom_det","label","score","det_obj_img_name","iou","binary_label","description"])
  #Wykryj obiekty za pomocą DETR
  df = detect_objects(row['img_name'])
  sum_binary_label = 0
  #jeśli wykryty został co najmniej jeden obiekt
  if len(df.index)>0:
    for i, r in df.iterrows():
      #opisz obiekt za pomocą VisiionEncoderDecoder
      desc = predict_step([r["det_obj_img_name"]])
      #policz IoU z ground truth
      iou_val = calculate_iou(
          (row["left"], row["top"], row["right"], row["bottom"]),
          (r["left"], r["top"], r["right"], r["bottom"]))

      binary_label = 0
      #oceń, czy opis zaiwera odpowiedź na pytanie
      text_label = classify_qs(row["question"], desc, gpt_model, gpt_tokenizer, sf)
      if text_label == "yes":
        binary_label = 1
        sum_binary_label += binary_label
        #dodaj wartości do df
        new_row = pd.DataFrame({
            'image':[row["image"]],
            'width':[row["width"]],
            'height':[row["height"]],
            'left':[row["left"]],
            'top':[row["top"]],
            'right':[row["right"]],
            'bottom':[row["bottom"]],
            'question':[row["question"]],
            'img_name':[row["img_name"]],
            'left_det':[r["left"]],
            'top_det':[r["top"]],
            'right_det':[r["right"]],
            'bottom_det':[r["bottom"]],
            'label':[r["label"]],
            'score':[r["score"]],
            'det_obj_img_name':[r["det_obj_img_name"]],
            "iou":[iou_val],
            "binary_label":[binary_label],
            "description":[desc]
            })
        det_tmp_df = pd.concat([det_tmp_df, new_row], ignore_index=True)

  if sum_binary_label == 0:
    desc = ""
    iou_val = 0
    binary_label = 0
    left_det = 0
    right_det = 0
    top_det = 0
    bottom_det = 0
    score = 0
    label = ""
    new_row = pd.DataFrame({
          'image':[row["image"]],
          'width':[row["width"]],
          'height':[row["height"]],
          'left':[row["left"]],
          'top':[row["top"]],
          'right':[row["right"]],
          'bottom':[row["bottom"]],
          'question':[row["question"]],
          'img_name':[row["img_name"]],
          'left_det':[left_det],
          'top_det':[top_det],
          'right_det':[right_det],
          'bottom_det':[bottom_det],
          'label':[label],
          'score':[score],
          'det_obj_img_name':[row["img_name"]],
          "iou":[iou_val],
          "binary_label":[binary_label],
          "description":[desc]
          })
    det_tmp_df = pd.concat([det_tmp_df, new_row], ignore_index=True)

  det_df = pd.concat([det_df, det_tmp_df], ignore_index=True)
  remove_img(DEST_IMG_FOLDER)

  if (index % LOG_EVERY_N) == 0:
    print(f"Image {index} ({row['img_name']}) has been processed.")

det_df.to_csv(f"/content/val_desc.csv", index=False)
det_df.to_csv(f"{VAL_FOLDER}/val_desc.csv", index=False)
files.download(f"/content/val_desc.csv")

In [ ]:
test = pd.read_csv("/content/val_desc.csv")

In [ ]:
test[test.question == "what do we throw and catch in the air?"]

,image,width,height,left,top,right,bottom,question,img_name,left_det,top_det,right_det,bottom_det,label,score,det_obj_img_name,iou,binary_label,description
8467,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,417,424,33,527,62,what do we throw and catch in the air?,000000138713.jpg,274.02,229.46,428.06,416.17,person,0.989,/content/gdrive/MyDrive/Colab Notebooks/PW PRO...,0.000000,1,['a man holding a tennis ball in his hand']
8468,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,417,424,33,527,62,what do we throw and catch in the air?,000000138713.jpg,0.05,364.21,53.76,417.02,person,0.998,/content/gdrive/MyDrive/Colab Notebooks/PW PRO...,0.000000,1,['a man standing in a field with a soccer ball']
8469,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,417,424,33,527,62,what do we throw and catch in the air?,000000138713.jpg,424.12,32.40,527.93,64.09,frisbee,1.000,/content/gdrive/MyDrive/Colab Notebooks/PW PRO...,0.905959,1,['a green and white object in the air']
8470,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,417,424,33,527,62,what do we throw and catch in the air?,000000138713.jpg,60.79,123.19,363.21,414.04,person,1.000,/content/gdrive/MyDrive/Colab Notebooks/PW PRO...,0.000000,1,['a man in a green shirt is holding a frisbee']
8471,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,417,424,33,527,62,what do we throw and catch in the air?,000000138713.jpg,278.87,178.33,597.76,414.93,person,1.000,/content/gdrive/MyDrive/Colab Notebooks/PW PRO...,0.000000,1,['a man holding a frisbee while another man lo...


In [ ]:
test[test.question == "what do we throw and catch in the air?"].image.values

array(['https://toloka-cdn.azureedge.net/wsdmcup2023/000000138713.jpg',
       'https://toloka-cdn.azureedge.net/wsdmcup2023/000000138713.jpg',
       'https://toloka-cdn.azureedge.net/wsdmcup2023/000000138713.jpg',
       'https://toloka-cdn.azureedge.net/wsdmcup2023/000000138713.jpg',
       'https://toloka-cdn.azureedge.net/wsdmcup2023/000000138713.jpg'],
      dtype=object)

In [ ]:
all_iou = len(test.img_name)

In [ ]:
all_iou

8473

In [ ]:
sum_iou = test["iou"].sum()

In [ ]:
sum_iou

671.6917609165971

In [ ]:
avg_iou = sum_iou/all_iou*100

In [ ]:
print(f"Average IoU on test set: {avg_iou}")

Average IoU on test set: 7.9274372821503265
